This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

In [1]:
%cd /code

/code


---

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [2]:
from dotenv import load_dotenv
from notebook_helpers import usage_string, mprint

load_dotenv()

True

---

# Chains

A `chain` is an object that executes a sequence of tasks called `links`. Each link in the chain is a callable, which can be either a function or an object that implements the `__call__` method. **The output of one link serves as the input to the next link in the chain. So a `Chain` is a simple mechanism that takes an input and sends the input to the first link, and the propegates the output of the first link to the second link, and so on, until the end of the chain is reached, and returns the final result.** 

Furthermore, a chain aggregates the history (prompts/responses, token usage, costs, etc.) across all links. More specifically, it aggregates all of the `Record` objects across any link that has a `history` property (which returns a list of Record objects; a Record object contains the metadata of an event such as costs, tokens used, prompt, response, etc.). This functionality allows the chain to contain convenient properties that aggregate the costs and usage across all links of the chain, and can also be used to explore intermediate steps and events in the chain.

---

## Simple Example

Here's a simple example demonstrating llm-chain. A more in-depth example with additional explanation can be found below.

- Ask the chat model a question ("What is the meaning of life?")
- The model responds, and the response is sent to the next link, which creates and returns a new prompt indicating that the link's input (which is the output from the last link; i.e. the model's response) should be summarized.
- The new prompt is sent to the next link, which is the chat model, and the response is returned.

In [3]:
from llm_chain.base import Chain
from llm_chain.models import OpenAIChat

chat_model = OpenAIChat(model_name='gpt-3.5-turbo')
# each link is a callable where the output of one link is the input to the next link
chain = Chain(links=[
    chat_model,
    lambda x: f"Summarize the following in two sentences: ```{x}```",
    chat_model,
])
chain("What is the meaning of life?")

'The meaning of life is a philosophical question that has been debated for centuries, with different beliefs and interpretations across cultures. Some find meaning in happiness and fulfillment, while others seek it through religious or spiritual beliefs, ultimately making it a subjective and individual pursuit.'

In [4]:
# usage_string simply uses the properties in chain (chain.cost, chain.total_tokens, etc.)
# to create a formatted string
print(usage_string(chain))

Cost:              $0.00063
Total Tokens:       375
Prompt Tokens:      239
Response Tokens:    136



In [5]:
len(chain.history)

2

In [6]:
[type(x) for x in chain.history]

[llm_chain.base.ExchangeRecord, llm_chain.base.ExchangeRecord]

In [7]:
mprint("> " + chain.history[0].prompt)
mprint("> " + chain.history[0].response)
mprint("> " + chain.history[1].prompt)
mprint("> " + chain.history[1].response)

> What is the meaning of life?

> The meaning of life is a philosophical question that has been debated for centuries. Different people and cultures have different beliefs and interpretations. Some believe that the meaning of life is to seek happiness and fulfillment, while others find meaning in religious or spiritual beliefs. Ultimately, the meaning of life may be subjective and can vary from person to person. It is up to each individual to explore and find their own sense of purpose and meaning in life.

> Summarize the following in two sentences: ```The meaning of life is a philosophical question that has been debated for centuries. Different people and cultures have different beliefs and interpretations. Some believe that the meaning of life is to seek happiness and fulfillment, while others find meaning in religious or spiritual beliefs. Ultimately, the meaning of life may be subjective and can vary from person to person. It is up to each individual to explore and find their own sense of purpose and meaning in life.```

> The meaning of life is a philosophical question that has been debated for centuries, with different beliefs and interpretations across cultures. Some find meaning in happiness and fulfillment, while others seek it through religious or spiritual beliefs, ultimately making it a subjective and individual pursuit.

Notice in the exchange history above (which is sent to the OpenAI model), that in the second link (the line with the lambda function) we don't actually have to use the response (`x` in lambda) since it's already in the history. I simply did that for illustrative purposes. We could replace the second link with `lambda _: "Summarize your previous answer in two sentences."`, which ignores the ouput of the first link (i.e. first response from chat model) and would actually reduce the number of tokens we use since we aren't passing the previous response in the new exchange. Let's do that below.

In [8]:
from llm_chain.base import Chain
from llm_chain.models import OpenAIChat

chat_model = OpenAIChat(model_name='gpt-3.5-turbo')
chain = Chain(links=[
    chat_model,
    lambda _: "Summarize your previous answer in two sentences.",
    chat_model,
])
chain("What is the meaning of life?")

'The meaning of life is a subjective and philosophical question that has been debated for centuries. It varies from person to person and can be found in seeking happiness, fulfillment, or through religious or spiritual beliefs.'

In [9]:
print(usage_string(chain))

Cost:              $0.00048
Total Tokens:       278
Prompt Tokens:      152
Response Tokens:    126



As expected, the cost/token-usage is lower since we're not sending information that is already contained in the previous response. Compare the history below to the history above. 

In [10]:
mprint("> " + chain.history[0].prompt)
mprint("> " + chain.history[0].response)
mprint("> " + chain.history[1].prompt)
mprint("> " + chain.history[1].response)

> What is the meaning of life?

> The meaning of life is a philosophical question that has been debated for centuries. Different people and cultures have different beliefs and interpretations. Some believe that the meaning of life is to seek happiness and fulfillment, while others find meaning in religious or spiritual beliefs. Ultimately, the meaning of life may be subjective and can vary from person to person. It is up to each individual to explore and find their own sense of purpose and meaning in life.

> Summarize your previous answer in two sentences.

> The meaning of life is a subjective and philosophical question that has been debated for centuries. It varies from person to person and can be found in seeking happiness, fulfillment, or through religious or spiritual beliefs.

---

## In-depth Example

In the following example, we will ask a chat model a question, and provide the model with additional context based on the most relevant text we find in a web-search. For the sake of the example, we will ask the chat model to the answer it provides and summarize that answer.  The workflow is defined as follows:


|   input   |  output  |   link/task   |
|-----------|----------|----------|
|  `None` |   str    |  Ask a question: e.g. `"What is the meaning of life?"`  |
|    str    |   urls   |  Do a web-search via `DuckDuckGo` based on the question.  |
|   urls    | Documents|  Take the urls, scrape the corresponding web-pages, and convert to a list of `Document` objects  |
| Documents | Documents|  Split the `Document` objects into smaller chunks (also `Document` objects)  |
| Documents | `None` |  Store the Documents in a `Chroma` document index so that we can lookup the most relevant documents |
| `None`  |   str    |  Return the original question `What is the meaning of life?` |
|    str    |   str    |  Take the original question, lookup the most relevant documents, and construct the corresponding prompt with the documents injected into the prompt. |
|    str    |   str    |  Pass the prompt to the chat model; receive a response |
|    str    |   str    |  Construct a new prompt, containing the response and a request to summarize the response. |
|    str    |   str    |  Pass the new prompt to the chat model; receive a response |

**Notice how the output of one link matches the input of the next link. This means you could swap out any of the objects we use below with your own custom function or class and the only requirement is that the input/output matches for that particular link.**



### Initial objects in chain

Let's define the objects and functions we are going to include in the chain. We're including many links/tasks so that we can see how flexible our chain can be.

See the [tools.ipynb](https://github.com/shane-kercheval/llm-chain/tree/main/examples/tools.ipynb) notebook for examples of how the various helper classes below (e.g. `DuckDuckGoSearch`, `scrape_url`, etc.) are used.

#### Quick note on the `Value` object.

The only bit of magic below is that we're using a `Value` object to cache the initial question, feed it into the web-search, and then inject it back into the chain at a later point (passing it to the prompt-template). A `Value` object is a callable that, when called with a value caches and returns the same value, and when called without a value, simply returns the previously cached value. When you understand what it's doing, it's not magic at all. Here's a simple example:

In [11]:
from llm_chain.base import Value
cache = Value()
result = cache("This is a value")  # calling the object with a value caches and returns the value
print(result)  # the value in `result` is whatever was passed in to the Value object
print(cache())  # calling the object without a parameter returns the cached value
result = cache("new value")
print(result)
print(cache())

This is a value
This is a value
new value
new value


In [12]:
import re
from llm_chain.base import Document, Chain, Value
from llm_chain.models import OpenAIEmbedding, OpenAIChat
from llm_chain.tools import DuckDuckGoSearch, scrape_url, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.prompt_templates import DocSearchTemplate

# Seach DuckDuckGo based on initial question (passed into the chain)
duckduckgo_search = DuckDuckGoSearch(top_n=3)

# define a function that takes the links from the web-search, scrapes the web-pages,
# and then creates Document objects from the text of each web-page
def scrape_urls(search_results):
    """
    For each url (i.e. `href` in `search_results`):
    - extracts text
    - replace new-lines with spaces
    - create a Document object
    """
    return [
        Document(content=re.sub(r'\s+', ' ', scrape_url(x['href'])))
        for x in search_results
    ]

# Embeddings model used for document index/search (the documents created from the web-search)
embeddings_model = OpenAIEmbedding(model_name='text-embedding-ada-002')
document_index = ChromaDocumentIndex(embeddings_model=embeddings_model, n_results=3)
# DocSearchTemplate uses the ChromaDocumentIndex object to search for the most relevant documents
# (from the web-search) based on the intitial question (which it takes as an input)
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=3)
# Create a chat model without streaming.
non_streaming_chat = OpenAIChat(model_name='gpt-3.5-turbo')
# Create a chat model with streaming enabled via callback.
streaming_chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    streaming_callback=lambda x: print(x.response, end='|'),
)

# A `Value` object is a simple caching mechanism. It's a callable that, when passed a value, it
# caches and returns that value; and when called without a value, it returns the cached value.
# Below, it's being used to cache the original question, feed the question into the web-search
# (`DuckDuckGoSearch`), and then re-inject the question back in the chain and into the
# prompt-template (`DocSearchTemplate`).
question_1 = Value()
# This simple function takes the response from the original chat model and creates a prompt that
# asks the model to summarize the response.
question_2 = lambda x: f'Summarize the following in less than 20 words: "{x}"'

### Defining and Running the Chain

In [13]:
# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    question_1,
    duckduckgo_search,
    scrape_urls,
    split_documents,  # split web-pages into smaller chunks; defaults to chunk-size of 500
    document_index,  # __call__ function calls add() if given a list of documents (which is returned by `split_documents`)
    question_1,
    prompt_template,
    non_streaming_chat,
    question_2,
    streaming_chat,
])
# the value passed into `chain()` is passed to the `initial_question` object (which
# is `Value` object and so it caches the value passed in and also returns it) which then gets
# passed to the `duckduckgo_search` object, and so on.
# the response of the final model is streamed, because our chat model has the streaming_callback
# set, but it should also match the response returned
response = chain("What is the meaning of life?")

The| meaning| of| life| is| a| debated| philosophical| question| with| no| definitive| answer| due| to| different| perspectives| and| interpretations|.|

In [14]:
mprint("> " + response)

> The meaning of life is a debated philosophical question with no definitive answer due to different perspectives and interpretations.

---

### Total costs and usage

The `Chain` object aggregates the costs/usage across all links that have a `history` property where that `history` property returns `UsageRecord` objects. In other words, any link that tracks its own history automatically gets counted towards the total usage within the Chain.

In [15]:
# usage_string simply uses the properties in chain (chain.cost, chain.total_tokens, etc.)
# to create a formatted string
print(usage_string(chain))

Cost:              $0.00449
Total Tokens:       37,515
Prompt Tokens:      432
Response Tokens:    70
Embedding Tokens:   37,013



In [16]:
chain.embedding_tokens

37013

### History

Similar to tracking the costs/usage, we can dig into the history at a more granular level.

As you can see below, the `history` property returns a list containing:

- a `SearchRecord` object capturing our original search query and results
- two `EmbeddingsRecord` records; the first corresponds to getting the embeddings of all of the chunks from the web-pages; the second corresponds to getting the embedding of our original question so that we can find the most relevant chunks; the `EmbeddingsRecord` is a `UsageRecord` so it contains costs/usage
- two `MessageRecord` records; the first corresponds to the first question (prompt & response) we made to the chat model; the second corresponds to our second query to the chat model asking it to summarize the first response; the `MessageRecord` is a `UsageRecord` so it contains costs/usage

In [17]:
[type(x) for x in chain.history]

[llm_chain.tools.SearchRecord,
 llm_chain.base.EmbeddingRecord,
 llm_chain.base.EmbeddingRecord,
 llm_chain.base.ExchangeRecord,
 llm_chain.base.ExchangeRecord]

In [18]:
print(chain.history[3])
mprint('---')
print(chain.history[4])

timestamp: 2023-07-05 01:42:24.152; prompt: "Answer the question at the end of the text as trut..."; response: "The meaning of life is a philosophical question th...";  cost: $0.000628; total_tokens: 402; prompt_tokens: 352; response_tokens: 50; uuid: cd024d27-3e16-4b5d-ac27-21e049598645


---

timestamp: 2023-07-05 01:42:24.985; prompt: "Summarize the following in less than 20 words: "Th..."; response: "The meaning of life is a debated philosophical que...";  cost: $0.000160; total_tokens: 100; prompt_tokens: 80; response_tokens: 20; uuid: 776fd927-4f13-401b-af61-d22154f76fcb


#### `exchange_history`

We can use the `exchange_history` property which simply returns all of the history items that are of type `ExchangeRecord`

In [19]:
len(chain.exchange_history)

2

In [20]:
print(chain.exchange_history[0])  # same as chain.history[3]
mprint('---')
print(chain.exchange_history[1])  # same as chain.history[4]

timestamp: 2023-07-05 01:42:24.152; prompt: "Answer the question at the end of the text as trut..."; response: "The meaning of life is a philosophical question th...";  cost: $0.000628; total_tokens: 402; prompt_tokens: 352; response_tokens: 50; uuid: cd024d27-3e16-4b5d-ac27-21e049598645


---

timestamp: 2023-07-05 01:42:24.985; prompt: "Summarize the following in less than 20 words: "Th..."; response: "The meaning of life is a debated philosophical que...";  cost: $0.000160; total_tokens: 100; prompt_tokens: 80; response_tokens: 20; uuid: 776fd927-4f13-401b-af61-d22154f76fcb


#### First prompt to the chat model

You can see below that we used three chunks from our original web-search in the first prompt we sent to ChatGPT.

In [21]:
mprint(chain.exchange_history[0].prompt)


Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
What is the meaning of life? | GotQuestions.org New Top 20 Random Podcast Subscribe New Top 20 Random Podcast The Gospel About Us Ask Crucial Questions Content Index Question of the Week International Donate Subscribe LightDark Bible VersionESVCSBKJVNASBNIVNKJVNLT Font FamilyDefaultArialVerdanaHelveticaTahomaGeorgiaTimes New Roman Line HeightDefault1.01.21.52 Font SizeDefault1.01.21.52 Facebook Twitter Pinterest Email Text/SMS Find Out How to go to Heaven How to get right with God Home Content

meaning of life? Subscribe to the Question of the Week Get our Question of the Week delivered right to your inbox! Follow Us: Contact Us Citation Serve with Us About Us © Copyright 2002-2023 Got Questions Ministries. All rights reserved. Privacy Policy This page last updated: September 12, 2022

"leap", arguing that life is full of absurdity, and one must make his and her own values in an indifferent world. One can live meaningfully (free of despair and anxiety) in an unconditional commitment to something finite and devotes that meaningful life to the commitment, despite the vulnerability inherent to doing so.[87] Arthur Schopenhauer answered: "What is the meaning of life?" by stating that one's life reflects one's will, and that the will (life) is an aimless, irrational, and painful
```

Here is the question:

```
What is the meaning of life?
```


#### First response from the chat model

In [22]:
mprint("> " + chain.exchange_history[0].response)

> The meaning of life is a philosophical question that has been debated by various thinkers throughout history. Different individuals and schools of thought have offered different perspectives and interpretations on this topic. Therefore, there is no definitive or universally agreed-upon answer to the question.

#### Second prompt to the chat model

In [23]:
mprint(chain.exchange_history[1].prompt)

Summarize the following in less than 20 words: "The meaning of life is a philosophical question that has been debated by various thinkers throughout history. Different individuals and schools of thought have offered different perspectives and interpretations on this topic. Therefore, there is no definitive or universally agreed-upon answer to the question."

#### Second response from the chat model

In [24]:
mprint("> " + chain.exchange_history[1].response)

> The meaning of life is a debated philosophical question with no definitive answer due to different perspectives and interpretations.

---